# Preparação do estado de bit-flip

In [1]:
from sympy import cos, sin, sqrt, pi, Matrix, Symbol, exp, print_latex, simplify
#from sympy.physics.quantum import TensorProduct, Dagger
import numpy as np
from numpy import linspace
#import matplotlib.pyplot as plt
#import math
from theoric.tools import *
#import torch
from torch import tensor

theta = Symbol('theta',real=True)
phi = Symbol('phi',real=True)
gamma = Symbol('gamma',real=True, positive=True)
p = Symbol('p',real=True, positive=True)



def calculated_rho_AB(theta, phi, p):
    state = Matrix([[(sqrt(1-p)*cos(theta/2)),
                    (sqrt(p)*cos(theta/2)),
                    (sqrt(1-p)*exp(1j*phi)*sin(theta/2)),
                    -sqrt(p)*exp(1j*phi)*sin(theta/2)]])
    M_numpy = np.array(state.tolist(), dtype=np.complex64)
    return M_numpy
    return state

def get_target_op(func_estado):
    state1 = simplify(func_estado)
    state2 = np.zeros(4,dtype=complex)
    aux = 0
    for i in state1[0]:
        state2[aux] = i
        aux += 1
    target_op = np.outer(state2.conj(), state2)
    target_op = tensor(target_op)
    return target_op

#state = calculated_rho_AB(theta, phi, p)
state = calculated_rho_AB(pi/2, 0, 0.5)
state

array([[ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j]], dtype=complex64)

In [2]:
a = get_target_op(calculated_rho_AB(pi/2, 0, 1))
a


tensor([[ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.5000+0.j,  0.0000+0.j, -0.5000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [ 0.0000-0.j, -0.5000-0.j,  0.0000-0.j,  0.5000+0.j]],
       dtype=torch.complex128)

In [7]:
'''                                  ReadMe                                                      '''
'''# Nessa célula vamos apenas mudar a função geradora do estado a ser preparado                 '''
'''# Basta alterar a função que gera o estado, nesse caso target_op = init_state_ref(n_qubits, p)'''

from src.vqa_tools import general_vqacircuit_penny 
from src.generalize import *
from torch import tensor
from numpy import sin,cos,sqrt,outer,zeros, pi
import cmath
import pickle
from theoric.bf_theoric import *

def run_calcs(list_p, epochs):
    coerencias_R = []
    coerencias_L = []
    pretrain = True
    count = 0
    n_qubits = 2
    depht = 3
    _, params, _, _ = start_things(n_qubits, depht)
    for p in list_p:
        print(f'{count} de {len(list_p)}')
        count += 1
        circuit, _ = general_vqacircuit_penny(params, n_qubits, depht)

        # defina o estado a ser preparado abaixo
        #------------------------------------------------------------
        #target_op = bpf(pi/2, 0, p)
        target_op = get_target_op(calculated_rho_AB(pi/2, 0, p))
        #------------------------------------------------------------
    
        qc, qr, params = optmize(epochs, n_qubits, circuit, params, target_op, pretrain, 120)
        pretrain = False
        rho = tomograph(qc, qr)
        print(rho)
        coerencias_L, coerencias_R = results(rho, coerencias_R, coerencias_L)
    mylist = [coerencias_L, coerencias_R]
    with open('data/bf/bf-coherences.pkl', 'wb') as f:
        pickle.dump(mylist, f)
    with open('data/bf/bf-best_params.pkl', 'wb') as f:
        pickle.dump(params, f)
    plot_theoric_bf(list_p)
    plots(list_p, coerencias_R, coerencias_L)
    #save = [list_p, coerencias_R, coerencias_L]
    #with open('data/BPFlist_p-coerencias_R-coerencias_L.pkl', 'wb') as f:
    #    pickle.dump(save, f)
    

In [8]:
list_p = np.linspace(0,1,21)
run_calcs(list_p, 85)

0 de 21
[[ 4.94474664e-01+0.j          4.96761234e-01-0.00349451j
   2.31717868e-02+0.01430692j -1.25511452e-02+0.00702632j]
 [ 4.96761234e-01+0.00349451j  5.02093717e-01+0.j
   2.45510729e-02+0.0138672j  -1.13327125e-02+0.00628935j]
 [ 2.31717868e-02-0.01430692j  2.45510729e-02-0.0138672j
   2.27512899e-03+0.j          3.71433461e-04+0.00067686j]
 [-1.25511452e-02-0.00702632j -1.13327125e-02-0.00628935j
   3.71433461e-04-0.00067686j  1.15649021e-03+0.j        ]]
1 de 21
[[ 0.45939415+0.j          0.47070744-0.02374841j  0.09310708-0.02024371j
  -0.12839732-0.00074379j]
 [ 0.47070744+0.02374841j  0.48434305+0.j          0.09677257-0.0164379j
  -0.13134326-0.00772134j]
 [ 0.09310708+0.02024371j  0.09677257+0.0164379j   0.02020999+0.j
  -0.0257184 -0.00582655j]
 [-0.12839732+0.00074379j -0.13134326+0.00772134j -0.0257184 +0.00582655j
   0.0360528 +0.j        ]]
2 de 21
[[ 0.46958138+0.j          0.44560626-0.03612269j  0.14574112-0.00588501j
  -0.1660661 +0.00981636j]
 [ 0.44560626+0.036

In [4]:
list_p = np.linspace(0,1,21)
run_calcs(list_p, 85)

0 de 21


KeyboardInterrupt: 